In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division
%matplotlib inline 

In [ ]:
mDIR ='/home/lindb/teakettle/postmoran/all_tfk'
if not op.exists(mDIR):
    mkdir(mDIR)

# for all of teakettle

### Put into R
```R
source('~/imports.R')
library(PopGenReport)
library(adegenet)
cd('~/fragment/postmoran/final')

### get data
moran     <- read.csv('~/Desktop/moran_infile_final_jan_wconecounts.txt',sep='\t')
simpmoran <- moran[seq(1,nrow(moran),2),] # for spatial stuff

### name some stuff
chloro <- c('pc10','pt71936','pt87268')
nuc    <- c('rps12','rps2','rps39','rps50')
loci   <- c(nuc,chloro)

### make a blank df to fill in later
popgen <- matrix(0,ncol=len(loci)+1,nrow=luni(moran$samp))
popgen <- data.frame(popgen)
colnames(popgen) <- c('pop',loci)
rownames(popgen) <- uni(moran$samp)
head(popgen)

### get genotype
getgt = function(type,df,locus){
    rownames(df) <- 1:nrow(df)
    stopifnot(nrow(df)==1)
    if (type == "nuc"){
        one = df[1,locus]
        two = df[1,sprintf("%s_2",locus)]
        combined = sprintf("%s:%s",one,two)
    } else {
        combined = sprintf("%s:%s",df[1,locus],df[1,locus])
    }
    return(combined)
}

# make function to make df
getdf = function(treatment='tx'){
    if (!treatment %in% c('tx','plot','tfk')){
        stop(cat(sprintf("treatment %s not in %s",treatment,c('tx','plot','single'))))
    }
    # make blank df
    popgen <- matrix(0,ncol=len(loci)+1,nrow=luni(moran$samp))
    popgen <- data.frame(popgen)
    colnames(popgen) <- c('pop',loci)
    rownames(popgen) <- uni(moran$samp)

    # fill in blank df
    for (samp in uni(moran$samp)){
        df <- moran[which(moran$samp == samp),]

        plot = strsplit(samp,split="_")[[1]][1]
        tx   = substr(plot,1,2)
        if (treatment=='tx'){
            popgen[samp,'pop'] <- tx
        } else if (treatment == 'plot'){
            popgen[samp,'pop'] <- plot
        } else {
            popgen[samp,'pop'] <- 'teakettle'
        }
        for (locus in loci){
            type = NULL
            if (locus %in% nuc){
                type = 'nuc'
            } else {
                type = 'chloro'}
            if (df[1,locus] != 0){
                popgen[samp,locus] = getgt(type,df[1,],locus)
            } else if (df[2,locus] != 0) {
                popgen[samp,locus] = getgt(type,df[2,],locus)
            } else {
                popgen[samp,locus] = NA}}}
    return(popgen)}

### make dfs with either plot or treatment
popgen.tfk  <- getdf(treatment='tfk')  # one single pop
pg.tfk <- popgen.tfk[,names(popgen.tfk[2:len(names(popgen.tfk))])]
head(pg.tfk)

### convert to class(genind)
tfkpop  <- popgen.tfk$pop
gentfk  <- df2genind(pg.tfk,ploidy=2,sep=":",pop=tfkpop)


### calc gen dist
gd <- gd.smouse(gentfk)
gdmat <- as.matrix(gd)
dim(gdmat)
gdmat[1:5,1:5]

### get euclidian distance
## for popgenreport()
xy = matrix(NA,nrow=nrow(simpmoran),ncol=2)
for (i in 1:nrow(simpmoran)) {
    xy[i,1] <- simpmoran[i,'xutm']
    xy[i,2] <- simpmoran[i,'yutm']
}
gentfk@other$xy <- xy
# pgr <- popgenreport(gentfk,mk.spautocor=T,mk.pdf=F,mk.gd.smouse=T,foldername='pgrfiles',
#     fname='pgrout',path.pgr='~/fragment/postmoran')
# missing something to make gdsmouse
# m2 <- as.matrix(gdsmouse['geogdist'][[1]])
# m <- gdsmouse['geneticdist']
# m <- m[[1]]
# m <- as.matrix(m)
# dim(m)

## for from-scratch
distmat <- matrix(NA,ncol=nrow(simpmoran),nrow=nrow(simpmoran))
for (i in 1:nrow(simpmoran)){
    for (j in 1:nrow(simpmoran)){
        if (i > j){
            xutm.i = simpmoran[i,'xutm']
            yutm.i = simpmoran[i,'yutm']
            xutm.j = simpmoran[j,'xutm']
            yutm.j = simpmoran[j,'yutm']
            # if (rownames(distmat)[i] == as.character(i)){
            #     rownames(distmat)[i] <- simpmoran[i,'samp']
            # }
            # if (colnames(distmat)[j] == as.character(j)){
            #     colnaames(distmat)[j] <- simpmoran[j,'samp']
            # }
            distmat[i,j] <- sqrt(((xutm.i-xutm.j)^2) + ((yutm.i-yutm.j)^2))
            distmat[j,i] <- distmat[i,j]
        }
        # if (i == j){
        #     distmat[i,j] <- 0
        # }
    }
}
diag(distmat) <- 0
distmat <- data.frame(distmat)
distmat[1:5,1:5]
rownames(distmat) <- simpmoran[,'samp']
colnames(distmat) <- simpmoran[,'samp']
dmat <- distmat[which(rownames(distmat) %in% rownames(gdmat)),which(colnames(distmat) %in% colnames(gdmat))]
all(rownames(dmat) == rownames(gdmat))
all(colnames(dmat) == colnames(gdmat))
luni(rownames(dmat)) == nrow(gdmat)
dim(dmat) == dim(gdmat)
#saveRDS(distmat,fname(getwd(),'distmat.RDS'))


### get spatial autocorrelation
bins = ceiling(.max(dmat)/10) # = 173
sauto <- spautocor(gdmat,dmat,bins = 173)

### get a better plot than stupid pgr
# auto.tfk <- pgr['spaautocor'][[1]][[1]]
# plot(auto.tfk$bin,auto.tfk$r)




### save environment
rds(,erase=TRUE)
```

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/all_tfk/final'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# copy from desktop to godel
# scp ~/fragment/postmoran/final/*RDS godel:/home/lindb/teakettle/postmoran/all_tfk/final

### multiprocess null confidence intervals for spatial autocorrelation

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/all_tfk/null/rfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/all_tfk/null/shfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
if not op.exists('/home/lindb/teakettle/postmoran/all_tfk/null/outfiles/'):
    mkdir('/home/lindb/teakettle/postmoran/all_tfk/null/outfiles/')

In [ ]:
for i in range(999):
    text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/all_tfk/final')

# load dependencies
getrds()

spautocor <- function (gen.m, eucl.m, shuffle = FALSE, bins = 10) 
{
    gd <- gen.m
    ed <- eucl.m
    if (shuffle == TRUE) {
        gdd <- as.dist(gd)
        gdsample <- sample(1:length(gdd), length(gdd))
        gd[lower.tri(gd)] <- gdd[gdsample]
        gd[upper.tri(gd)] <- gdd[gdsample]
        diag(gd) <- 0
    }
    cdmat <- function(gd) {
        dimen <- nrow(gd)
        sgd <- sum(gd, na.rm = TRUE)
        cscd <- matrix(colSums(gd, na.rm = TRUE), dimen, dimen)
        rscd <- matrix(rowSums(gd, na.rm = TRUE), dimen, dimen, 
            byrow = TRUE)
        cd <- 0.5 * (-gd + 1/dimen * (cscd + rscd) - 1/dimen^2 * 
            (sgd))
        cd
    }
    cd <- cdmat(gd)
    ed[upper.tri(ed)] <- NA
    diag(ed) <- NA
    r <- NA
    distance <- NA
    N <- NA
    steps <- signif(diff(range(ed, na.rm = TRUE))/bins, 4)
    for (d in 1:bins) {
        index <- which(ed <= d * steps & ed > (d - 1) * steps, 
            arr.ind = TRUE)
        cx <- sum(cd[index])
        cxii <- sum(diag(cd)[index[, 1]])
        cxjj <- sum(diag(cd)[index[, 2]])
        r[d] <- 2 * cx/(cxii + cxjj)
        distance[d] <- steps * d
        N[d] <- length(index)
    }
    if (shuffle == FALSE) 
        res <- data.frame(bin = distance, N = N, r = r)
    else res <- data.frame(r = r)
    res
}

# get null value
shuff <- spautocor(gdmat,dmat,bins=bins,shuffle=T)
filE = sprintf('/home/lindb/teakettle/postmoran/all_tfk/null/outfiles/shuff_%%s.RDS',sprintf('%%03d',%s))
saveRDS(shuff,filE)
''' % (i)
    filE = '/home/lindb/teakettle/postmoran/all_tfk/null/rfiles/null_%s.R' % str(i).zfill(3)
    with open(filE,'w') as o:
        o.write("%s" % text)
    o.close()

In [ ]:
files = fs('/home/lindb/teakettle/postmoran/all_tfk/null/rfiles/')
math.ceil(len(files)/195)

In [ ]:
files = fs('/home/lindb/teakettle/postmoran/all_tfk/null/rfiles/')

fcount  = 0
shcount = 0
rcount  = 0
for f in files:
    num = op.basename(f).split(".R")[0].split("_")[1]
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N shuff%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (num,
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == 6 or fcount == len(files):
        filE = '/home/lindb/teakettle/postmoran/all_tfk/null/shfiles/shuff_%s.sh' % str(shcount).zfill(3)
        with open(filE,'w') as o:
            o.write("%s" % text)
        rcount = 0
        shcount += 1

### combine null runs

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/all_tfk/null/outfiles/'
text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/all_tfk/')

sauto <- readRDS('/home/lindb/teakettle/postmoran/all_tfk/final/sauto.RDS')

# get a list of all of the null runs
DIR = '/home/lindb/teakettle/postmoran/all_tfk/null/outfiles/'
R <- list()
for (f in list.files(DIR,full.names=T,pattern='.RDS')){
    r <- readRDS(f)
    R <- append(R,list(r))
    names(R)[len(R)] <- as.character(len(R))
}
r <- data.frame(sauto$r)
names(r) <- 'r'
R <- append(R,r)
names(R)[1000] <- as.character(1000)

# get a list for each distance class, appending values from 1000 null runs
Rm <- list()
for (i in 1:173){
    print(i)
    lst <- c()
    for (j in 1:len(R)){
        lst <- c(lst, R[[as.character(j)]][i,'r'])
    }
    Rm <- append(Rm,list(lst))
}

# get 2.5th and 97.5th quantiles
null <- list()
for (i in 1:len(Rm)){
    low = sort(Rm[[i]])[25]
    high = sort(Rm[[i]])[975]
    null <- append(null,list(c(low,high)))
}


filE = fname(getwd(),'full_geno_null.RDS')
saveRDS(null,filE)
filE = fname(getwd(),'full_geno_R.RDS')
saveRDS(R,filE)
filE = fname(getwd(),'full_geno_Rm.RDS')
saveRDS(Rm,filE)
'''
filE = '/home/lindb/teakettle/postmoran/all_tfk/combine_R_vector_full_geno.R'
with open(filE,'w') as o:
    o.write("%s" % text)
o.close()

In [ ]:
#! cd /home/lindb/teakettle/postmoran/all_tfk/
#! R --no-save < /home/lindb/teakettle/postmoran/all_tfk/combine_R_vector.R

In [ ]:
# copy files to local
# scp godel:/home/lindb/teakettle/postmoran/all_tfk/size_class/adultxadult/full_geno_null.RDS ~/Downloads/all_tfk-adultxadult

# plot spatial autocorrelation
### put into R
```R
source('~/imports.R')
cd('~/Downloads/')

### load Rm, R, and sauto
# null <- readRDS('/home/lindb/teakettle/postmoran/all_tfk/full_geno_null.RDS')
# sauto <- readRDS('/home/lindb/teakettle/postmoran/all_tfk/final/sauto.RDS')

getrds()

# get quantiles for each distance class
lows <- c()
his <- c()
for (i in 1:len(null)){
	lows <- c(lows,null[[i]][1])
	his <- c(his,null[[i]][2])
}

plot(sauto$bin,sauto$r,type="l",xlab="Distance (meters)",ylab="r",ylim=c(-0.13,0.13),xaxt="n")
xtick <- seq(0,max(sauto$bin),100)
axis(side=1,at=xtick)
#axis(side=1,at=xtick,labels=FALSE)
#text(x=xtick,par("usr")[3],labels=xtick
polygon(c(sauto$bin,rev(sauto$bin)),c(lows,rev(his)),col='grey75',border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
```

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/figs'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# scp ~/Desktop/all_tfk_full_geno.pdf godel:/home/lindb/teakettle/postmoran/figs

In [ ]:
?WImage

In [ ]:
from wand.image import Image as WImage
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/all_tfk_full_geno.pdf')
img

# spatial autocor by plot - all genotypes

### put into R
```R
source('~/imports.R')
library(PopGenReport)
library(adegenet)
cd('~/fragment/postmoran/final')

### load dependencies
getrds()
popgen.plot <- getdf(treatment='plot') # plot-level pop
pg.plot <- popgen.plot[,colnames(popgen.plot)[2:len(colnames(popgen.plot))]]

### get a dataframe for each plot
plots <- uni(popgen.plot$pop)
for (plot in plots){
	df <- popgen.plot[which(popgen.plot$pop == plot),]
	assign(sprintf("%spop",plot),df$pop)
	df <- df[,colnames(df)[2:len(colnames(df))]]
	p <- plot
	assign(p,df)
}

### get genind objects
geninds <- c()
for (plot in plots){
	df      <- get(plot)
	pops    <- get(sprintf("%spop",plot))
	genind  <- sprintf("gen%s",plot)
	geninds <- c(geninds,genind)
	assign(genind,df2genind(df,ploidy=2,sep=":",pop=pops))
}

### get gd objects
gdnames = c()
for (genind in geninds){
	gdname <- sprintf("gd%s",genind)
	gdnames <- c(gdnames,gdname)
	gd <- gd.smouse(get(genind))
	gd <- as.matrix(gd)
	assign(gdname,gd)
}

### get distmats
dim(distmat) # 1362x1362
distmats <- c()
for (gd in gdnames){
	gdist <- get(gd)
	samps <- rownames(gdist)
	d <- distmat[which(rownames(distmat) %in% samps),which(colnames(distmat) %in% samps)]
	stopifnot(all(dim(d) == dim(get(gd))))
	distname <- sprintf("dist%s",gd)
	distmats <- c(distmats,distname)
	assign(distname,d)
}

### get spatial autocorrelation
autos <- c()
autobins <- c()
for (gd in gdnames){
	print(gd)
	gdist <- get(gd)
	d <- get(sprintf("dist%s",gd))
	bins = ceiling(.max(d)/10)
	binname = sprintf("bin%s",gd)
	autobins <- c(autobins,binname)
	auto <- spautocor(gdist,d,bins=bins)
	autoname <- sprintf("auto%s",gd)
	autos <- c(autos,autoname)
	assign(autoname,auto)
}

### save environment
cd('/Users/Brandon1/fragment/postmoran/by_plot')
rds()
```

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_plot/final'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# copy files to cluster
# scp /Users/Brandon1/fragment/postmoran/by_plot/*RDS godel:/home/lindb/teakettle/postmoran/by_plot/final

In [ ]:
dirs = ['/home/lindb/teakettle/postmoran/by_plot/final',
        '/home/lindb/teakettle/postmoran/by_plot/bc1',
        '/home/lindb/teakettle/postmoran/by_plot/bc2',
        '/home/lindb/teakettle/postmoran/by_plot/bc3',
        '/home/lindb/teakettle/postmoran/by_plot/uc1',
        '/home/lindb/teakettle/postmoran/by_plot/uc2',
        '/home/lindb/teakettle/postmoran/by_plot/uc3',
        '/home/lindb/teakettle/postmoran/by_plot/un1',
        '/home/lindb/teakettle/postmoran/by_plot/un2',
        '/home/lindb/teakettle/postmoran/by_plot/un3',
        '/home/lindb/teakettle/postmoran/by_plot/us3']
for d in dirs:
    if not op.exists(d):
        mkdir(d)

In [ ]:
# make some dirs
for d in dirs:
    if not 'final' in d:
        dnames = []
        dnames.append(op.join(d,'outfiles'))
        dnames.append(op.join(d,'rfiles'))
        dnames.append(op.join(d,'shfiles'))
        dnames.append(op.join(d,'null'))
        for D in dnames:
            if not op.exists(D):
                mkdir(D)

In [ ]:
# get plot IDs
plots = []
for d in dirs:
    if not 'final' in d:
        plots.append(op.basename(d))
plots

In [ ]:
# write scripts per plot to get 999 null values of spatial autocorrelation
for plot in plots:
    for i in range(999):
        text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_plot/final/')

# load dependencies
getrds()

spautocor <- function (gen.m, eucl.m, shuffle = FALSE, bins = 10) 
{
    gd <- gen.m
    ed <- eucl.m
    if (shuffle == TRUE) {
        gdd <- as.dist(gd)
        gdsample <- sample(1:length(gdd), length(gdd))
        gd[lower.tri(gd)] <- gdd[gdsample]
        gd[upper.tri(gd)] <- gdd[gdsample]
        diag(gd) <- 0
    }
    cdmat <- function(gd) {
        dimen <- nrow(gd)
        sgd <- sum(gd, na.rm = TRUE)
        cscd <- matrix(colSums(gd, na.rm = TRUE), dimen, dimen)
        rscd <- matrix(rowSums(gd, na.rm = TRUE), dimen, dimen, 
            byrow = TRUE)
        cd <- 0.5 * (-gd + 1/dimen * (cscd + rscd) - 1/dimen^2 * 
            (sgd))
        cd
    }
    cd <- cdmat(gd)
    ed[upper.tri(ed)] <- NA
    diag(ed) <- NA
    r <- NA
    distance <- NA
    N <- NA
    steps <- signif(diff(range(ed, na.rm = TRUE))/bins, 4)
    for (d in 1:bins) {
        index <- which(ed <= d * steps & ed > (d - 1) * steps, 
            arr.ind = TRUE)
        cx <- sum(cd[index])
        cxii <- sum(diag(cd)[index[, 1]])
        cxjj <- sum(diag(cd)[index[, 2]])
        r[d] <- 2 * cx/(cxii + cxjj)
        distance[d] <- steps * d
        N[d] <- length(index)
    }
    if (shuffle == FALSE) 
        res <- data.frame(bin = distance, N = N, r = r)
    else res <- data.frame(r = r)
    res
}

# get null value
gdmat <- get('%s')
dmat  <- get('%s')
bins  <- get('%s')
shuff <- spautocor(gdmat,dmat,bins=bins,shuffle=T)
filE = sprintf('/home/lindb/teakettle/postmoran/by_plot/%s/outfiles/shuff_%%s.RDS',sprintf('%%03d',%s))
saveRDS(shuff,filE)
    ''' % ('gdgen%s' % plot,
           'distgdgen%s' % plot,
           'bingdgen%s' % plot,
           plot,
           i)
        filE = '/home/lindb/teakettle/postmoran/by_plot/%s/rfiles/null_%s.R' % (plot,str(i).zfill(3))
        with open(filE,'w') as o:
            o.write("%s" % text)
        o.close()

In [ ]:
# gather r files, make sure there are 999 for each plot
rfiles = []
for p in plots:
    DIR = '/home/lindb/teakettle/postmoran/by_plot/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)
len(rfiles)

In [ ]:
rfiles[:10]

In [ ]:
# how many r files per sh file
ceil = math.ceil(len(rfiles)/195)
ceil

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_plot/shfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# get .R files
rfiles = []
for p in plots:
    DIR = '/home/lindb/teakettle/postmoran/by_plot/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)

# write sh files
fcount  = 0
shcount = 0
rcount  = 0
for f in rfiles:
    num = str(shcount).zfill(3)
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N shuff%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (num,
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == ceil or fcount == len(rfiles):
        filE = '/home/lindb/teakettle/postmoran/by_plot/shfiles/shuff_%s.sh' % str(shcount).zfill(3)
        with open(filE,'w') as o:
            o.write("%s" % text)
        rcount = 0
        shcount += 1

In [ ]:
# qsub the sh files
#! cd /home/lindb/teakettle/postmoran/by_plot/shfiles/
#! ls /home/lindb/teakettle/postmoran/by_plot/shfiles/*sh | parallel qsub {}

### check on progress

In [ ]:
for p in plots:
    DIR = '/home/lindb/teakettle/postmoran/by_plot/%s/outfiles' % p
    assert(op.exists(DIR))
    print p
    print len(ls(DIR))

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_plot/%s/outfiles' % 'bc1'
sorted(ls(DIR))[1:10]

In [ ]:
DIR

In [ ]:
# redistribute jobs on cluster
shuffs = []
for p in plots:
    for i in range(999):
        outfile = '/home/lindb/teakettle/postmoran/by_plot/%s/outfiles/shuff_%s.RDS' % (p,str(i).zfill(3))
        if not outfile in fs('/home/lindb/teakettle/postmoran/by_plot/%s/outfiles/' % (p)):
            rfile = '/home/lindb/teakettle/postmoran/by_plot/%s/rfiles/null_%s.R' % (p,str(i).zfill(3))
            assert(op.exists(rfile))
            shuffs.append(rfile)
#         else:
#              print outfile
len(shuffs)

In [ ]:
DIR = "/home/lindb/teakettle/postmoran/by_plot/shfiles/remsh"
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# write shuff sh files
ceil = math.ceil(len(shuffs)/195)

fcount  = 0
shcount = 0
rcount  = 0
for f in shuffs:
    num = str(shcount).zfill(3)
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N shuff%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (num,
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == ceil or fcount == len(shuffs):
        filE = '/home/lindb/teakettle/postmoran/by_plot/shfiles/remsh/shuff_%s.sh' % str(shcount).zfill(3)
        with open(filE,'w') as o:
            o.write("%s" % text)
        rcount = 0
        shcount += 1

In [ ]:
## qsub sh files
# cd /home/lindb/teakettle/postmoran/by_plot/shfiles/remsh/
# ls *sh | parallel qsub {}

In [ ]:
# check on progress
for p in plots:
    DIR = '/home/lindb/teakettle/postmoran/by_plot/%s/outfiles' % p
    assert(op.exists(DIR))
#     print p
    assert len(ls(DIR)) == 999

### combine null runs for each plot

In [ ]:
for plot in plots:
    text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_plot/final')
getrds()
plot = '%s'

#sauto <- readRDS('/home/lindb/teakettle/postmoran/nuclear/final/sauto.RDS')
sauto <- get(sprintf('autogdgen%%s',plot))

# get a list of all of the null runs
DIR = sprintf('/home/lindb/teakettle/postmoran/by_plot/%%s/outfiles',plot)
R <- list()
for (f in list.files(DIR,full.names=T,pattern='.RDS')){
    r <- readRDS(f)
    R <- append(R,list(r))
    names(R)[len(R)] <- as.character(len(R))
}
r <- data.frame(sauto$r)
names(r) <- 'r'
R <- append(R,list(r))
names(R)[1000] <- as.character(1000)

# get a list for each distance class, appending values from 1000 null runs
Rm <- list()
bins <- get(sprintf('bingdgen%%s',plot))
for (i in 1:bins){
    print(i)
    lst <- c()
    for (j in 1:len(R)){
        lst <- c(lst, R[[as.character(j)]][i,'r'])
    }
    Rm <- append(Rm,list(lst))
}

# get 2.5th and 97.5th quantiles
null <- list()
for (i in 1:len(Rm)){
    low = sort(Rm[[i]])[25]
    high = sort(Rm[[i]])[975]
    null <- append(null,list(c(low,high)))
}

### save environment
cd(sprintf('/home/lindb/teakettle/postmoran/by_plot/%%s',plot))
filE = fname(getwd(),sprintf('%%s_full_geno_null.RDS',plot))
saveRDS(null,filE)
filE = fname(getwd(),sprintf('%%s_full_geno_R.RDS',plot))
saveRDS(R,filE)
filE = fname(getwd(),sprintf('%%s_full_geno_Rm.RDS',plot))
saveRDS(Rm,filE)

''' % (plot)
    filE = '/home/lindb/teakettle/postmoran/by_plot/%s/%s_combine_R_vector_full_geno.R' % (plot,plot)
    with open(filE,'w') as o:
        o.write("%s" % text)
    o.close()

In [ ]:
rfiles = []
for plot in plots:
    f = '/home/lindb/teakettle/postmoran/by_plot/%s/%s_combine_R_vector_full_geno.R' % (plot,plot)
    rfiles.append(f)

shfiles = []
for f in rfiles:
    plot = op.basename(op.dirname(f))
    text ='''#!/bin/bash
#$ -N comb%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (plot,op.dirname(f),f)
    DIR = '/home/lindb/teakettle/postmoran/by_plot/shfiles/%s' % plot
    if not op.exists(DIR):
        mkdir(DIR)
    filE = op.join(DIR,'%s.sh' % plot)
    shfiles.append(filE)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
for f in shfiles:
    !qsub $f

In [ ]:
shfiles

In [ ]:
plots

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_plot/necfiles' 
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# scp these files to computer
files = [f for f in fs('/home/lindb/teakettle/postmoran/by_plot/final/') if 'autogdgen' in f]
[files.append(f) for f in fs('/home/lindb/teakettle/postmoran/by_plot/final/') if 'bingdgen' in f]
for plot in plots:
    f = '/home/lindb/teakettle/postmoran/by_plot/%s/%s_full_geno_null.RDS' % (plot,plot)
    assert op.exists(f)
    files.append(f)

In [ ]:
files

### Put into R
```R
source('~/imports.R')
library(PopGenReport)
library(adegenet)
cd('~/Downloads/by_plot_full_geno')
getrds()
plots <- c('bc1','bc2','bc3','uc1','uc2','uc3','un1','un2','un3','us3')

Lows <- list()
His  <- list()
for (plot in plots){
    null <- get(sprintf('%s_full_geno_null',plot))
    h <- c()
    l <- c()
    for (i in 1:len(null)){
        h <- c(h,null[[i]][1])
        l <- c(l,null[[i]][2])
    }
    Lows <- append(Lows,list(l))
    His  <- append(His,list(h))
    names(Lows)[len(Lows)] <- plot
    names(His)[len(His)]   <- plot
}
```
### then use code below to plot figs

In [ ]:
# print out text to paste into R
for plot in plots:
    text = '''
plot = '%s'
his  <- His[[sprintf('%%s',plot)]]
his[which(is.na(his))] <- 0
lows <- Lows[[sprintf('%%s',plot)]]
lows[which(is.na(lows))] <- 0
sauto <- get(sprintf('autogdgen%%s',plot))
sauto[which(is.na(sauto$r)),'r'] <- 0
plot(sauto$bin,sauto$r,
    type="l",
    xlab="Distance (meters)",
    ylab="r",
    ylim=c(-0.13,0.13),
    xaxt="n",
    main=sprintf('%%s',plot))
xtick <- seq(0,max(sauto$bin),100)
axis(side=1,at=xtick)
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col='grey75',border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
''' % plot
    print text
    print '\n\n'

# spatial autocor by treatment
### combine treatement, ignore interplot distances

### put into R
```R
source('~/imports.R')
library(PopGenReport)
cd('~/fragment/postmoran/final')

### load dependencies
getrds()
popgen.tx <- getdf(treatment='tx')
pg.tx <- popgen.tx[,colnames(popgen.tx)[2:len(colnames(popgen.tx))]]

### get a dataframe for each plot
plots <- uni(popgen.tx$pop)
dfs <- c()
for (plot in plots){
	df <- popgen.tx[which(popgen.tx$pop == plot),]
	assign(sprintf("%spop",plot),df$pop)
	df <- df[,colnames(df)[2:len(colnames(df))]]
	p <- plot
	assign(p,df)
}

### get genind objects
geninds <- c()
for (plot in plots){
	df      <- get(plot)
	pops    <- get(sprintf("%spop",plot))
	genind  <- sprintf("gen%s",plot)
	geninds <- c(geninds,genind)
	assign(genind,df2genind(df,ploidy=2,sep=":",pop=pops))
}

### get gd objects
gdnames = c()
for (genind in geninds){
	gdname <- sprintf("gd%s",genind)
	gdnames <- c(gdnames,gdname)
	gd <- gd.smouse(get(genind))
	gd <- as.matrix(gd)
	assign(gdname,gd)
}
# adjust gd objects so that GDs between plots are NA - doing this gave spatial auto r-vals outside of [-1,1]
# for (k in 1:len(gdnames)){
# 	print(gdnames[k])
# 	gd <- get(gdnames[k])
# 	for (i in 1:len(rownames(gd)) ){
# 		for (j in 1:len(colnames(gd)) ){
# 			if (i > j){
# 				sampi = rownames(gd)[i]
# 				txi <- substr(sampi,1,3)
# 				sampj = colnames(gd)[j]
# 				txj = substr(sampj,1,3)
# 				if(txi!=txj){
# 					gd[i,j] <- gd[j,i] <- NA
# 				}
# 			}
# 		}
# 	}
# 	assign(gdnames[k],gd)
# }

### get euclidian distance
gettx <- function(num,kind='tx'){
	samp <- simpmoran[num,'samp']
	plot <- strsplit(samp,"_")[[1]][1]
	tx   <- substr(plot,1,2)
	if (kind == 'tx'){
		return(tx)
	} else {
		return(plot)
	}
}
# fill in NAs for distances between plots
head(distmat) # full distmat from 07a_spatial_auto...ipynb
for (i in 1:nrow(simpmoran)){
	txi <- gettx(i,kind='plot')
	for (j in 1:nrow(simpmoran)){
		txj <- gettx(j,kind='plot')
		if (i > j){
			if (!(txi == txj)){
				distmat[i,j] <- distmat[j,i] <- NA
			}
		}
	}
}
simpmoran[,'plot']      <- substr(simpmoran$samp,1,3)
simpmoran[,'treatment'] <- substr(simpmoran$plot,1,2)
simpmoran[,'plot_rep']  <- substr(simpmoran$plot,3,3)

# from full distmat, create smaller distmats based on treatment type
txs <- uni(simpmoran$treatment)
distnames <- c()
for (tx in txs){
	sampnames <- simpmoran[which(simpmoran[,'treatment']==tx),'samp']
	d <- distmat[which(rownames(distmat) %in% sampnames),which(colnames(distmat) %in% sampnames)]
	d <- d[which(rownames(d) %in% rownames(get(sprintf('gdgen%s',tx)))) ,
			which(colnames(d) %in% colnames(get(sprintf('gdgen%s',tx))))]
	print(dim(d))
	distname  <- sprintf('dist%s',tx)
	distnames <- c(distnames,distname)
	assign(distname,d)
}

### get spatial autocorrelation
autos <- c()
autobins <- c()
for (gd in gdnames){
	print(gd)
	gdist <- get(gd)
	tx <- substr(gd,6,7)
	d <- get(sprintf("dist%s",tx))
	bins = ceiling(.max(d)/10)
	binname = sprintf("bin%s",gd)
	autobins <- c(autobins,binname)
	auto <- spautocor(gdist,d,bins=bins)
	autoname <- sprintf("auto%s",gd)
	autos <- c(autos,autoname)
	assign(autoname,auto)
}

### save environment
DIR = '~/fragment/postmoran/by_tx/'
if (!(file.exists(DIR))){
	dir.create(DIR)
}
cd(DIR)
rds()
```

In [ ]:
# copy to godel
DIR = '/home/lindb/teakettle/postmoran/by_tx/final'
if not op.exists(DIR):
    mkdir(DIR)
# !scp ~/fragment/postmoran/by_tx/*RDS godel:/home/lindb/teakettle/postmoran/by_tx/final

### multiprocess null confidence intervals

In [ ]:
dirs = ['/home/lindb/teakettle/postmoran/by_tx/bc',
        '/home/lindb/teakettle/postmoran/by_tx/uc',
        '/home/lindb/teakettle/postmoran/by_tx/un',
        '/home/lindb/teakettle/postmoran/by_tx/us']
for d in dirs:
    if not op.exists(d):
        mkdir(d)

In [ ]:
# make some dirs
for d in dirs:
    if not 'final' in d:
        dnames = []
        dnames.append(op.join(d,'outfiles'))
        dnames.append(op.join(d,'rfiles'))
        dnames.append(op.join(d,'shfiles'))
        dnames.append(op.join(d,'null'))
        for D in dnames:
            if not op.exists(D):
                mkdir(D)

In [ ]:
# get plot IDs
txs = []
for d in dirs:
    if not 'final' in d:
        txs.append(op.basename(d))
txs

In [ ]:
# write scripts per tx to get 999 null values of spatial autocorrelation
for tx in txs:
    for i in range(999):
        text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_tx/final/')

# load dependencies
getrds()

spautocor <- function (gen.m, eucl.m, shuffle = FALSE, bins = 10) 
{
    gd <- gen.m
    ed <- eucl.m
    if (shuffle == TRUE) {
        gdd <- as.dist(gd)
        gdsample <- sample(1:length(gdd), length(gdd))
        gd[lower.tri(gd)] <- gdd[gdsample]
        gd[upper.tri(gd)] <- gdd[gdsample]
        diag(gd) <- 0
    }
    cdmat <- function(gd) {
        dimen <- nrow(gd)
        sgd <- sum(gd, na.rm = TRUE)
        cscd <- matrix(colSums(gd, na.rm = TRUE), dimen, dimen)
        rscd <- matrix(rowSums(gd, na.rm = TRUE), dimen, dimen, 
            byrow = TRUE)
        cd <- 0.5 * (-gd + 1/dimen * (cscd + rscd) - 1/dimen^2 * 
            (sgd))
        cd
    }
    cd <- cdmat(gd)
    ed[upper.tri(ed)] <- NA
    diag(ed) <- NA
    r <- NA
    distance <- NA
    N <- NA
    steps <- signif(diff(range(ed, na.rm = TRUE))/bins, 4)
    for (d in 1:bins) {
        index <- which(ed <= d * steps & ed > (d - 1) * steps, 
            arr.ind = TRUE)
        cx <- sum(cd[index])
        cxii <- sum(diag(cd)[index[, 1]])
        cxjj <- sum(diag(cd)[index[, 2]])
        r[d] <- 2 * cx/(cxii + cxjj)
        distance[d] <- steps * d
        N[d] <- length(index)
    }
    if (shuffle == FALSE) 
        res <- data.frame(bin = distance, N = N, r = r)
    else res <- data.frame(r = r)
    res
}

# get null value
gdmat <- get('%s')
dmat  <- get('%s')
bins  <- get('%s')
shuff <- spautocor(gdmat,dmat,bins=bins,shuffle=T)
filE = sprintf('/home/lindb/teakettle/postmoran/by_tx/%s/outfiles/shuff_%%s.RDS',sprintf('%%03d',%s))
saveRDS(shuff,filE)
    ''' % ('gdgen%s' % tx,
           'dist%s' % tx,
           'bingdgen%s' % tx,
           tx,
           i)
        filE = '/home/lindb/teakettle/postmoran/by_tx/%s/rfiles/null_%s.R' % (tx,str(i).zfill(3))
        with open(filE,'w') as o:
            o.write("%s" % text)
        o.close()

In [ ]:
# gather r files, make sure there are 999 for each plot
rfiles = []
for p in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)
len(rfiles)

In [ ]:
# how many r files per sh file
ceil = math.ceil(len(rfiles)/195)
ceil

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/shfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# get .R files
rfiles = [] # redundant so I don't use wrong rfiles
for p in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)

# write sh files
fcount  = 0
shcount = 0
rcount  = 0
for f in rfiles:
    num = str(shcount).zfill(3)
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N shuff%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (num,
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == ceil or fcount == len(rfiles):
        filE = '/home/lindb/teakettle/postmoran/by_tx/shfiles/shuff_%s.sh' % str(shcount).zfill(3)
        with open(filE,'w') as o:
            o.write("%s" % text)
        rcount = 0
        shcount += 1

In [ ]:
shfiles = fs('/home/lindb/teakettle/postmoran/by_tx/shfiles/')
# for f in shfiles:
#     !qsub $f

In [ ]:
# combine null runs into a couple objects
for tx in txs:
    text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_tx/final')
getrds()
tx = '%s'

#sauto <- readRDS('/home/lindb/teakettle/postmoran/nuclear/final/sauto.RDS')
sauto <- get(sprintf('autogdgen%%s',tx))

# get a list of all of the null runs
DIR = sprintf('/home/lindb/teakettle/postmoran/by_tx/%%s/outfiles',tx)
R <- list()
for (f in list.files(DIR,full.names=T,pattern='.RDS')){
    r <- readRDS(f)
    R <- append(R,list(r))
    names(R)[len(R)] <- as.character(len(R))
}
r <- data.frame(sauto$r)
names(r) <- 'r'
R <- append(R,list(r))
names(R)[1000] <- as.character(1000)

# get a list for each distance class, appending values from 1000 null runs
Rm <- list()
bins <- get(sprintf('bingdgen%%s',tx))
for (i in 1:bins){
    print(i)
    lst <- c()
    for (j in 1:len(R)){
        lst <- c(lst, R[[as.character(j)]][i,'r'])
    }
    Rm <- append(Rm,list(lst))
}

# get 2.5th and 97.5th quantiles
null <- list()
for (i in 1:len(Rm)){
    low = sort(Rm[[i]])[25]
    high = sort(Rm[[i]])[975]
    null <- append(null,list(c(low,high)))
}

### save environment
cd(sprintf('/home/lindb/teakettle/postmoran/by_tx/%%s',tx))
filE = fname(getwd(),sprintf('%%s_full_geno_null.RDS',tx))
saveRDS(null,filE)
filE = fname(getwd(),sprintf('%%s_full_geno_R.RDS',tx))
saveRDS(R,filE)
filE = fname(getwd(),sprintf('%%s_full_geno_Rm.RDS',tx))
saveRDS(Rm,filE)

''' % (tx)
    filE = '/home/lindb/teakettle/postmoran/by_tx/%s/%s_combine_R_vector_full_geno.R' % (tx,tx)
    with open(filE,'w') as o:
        o.write("%s" % text)
    o.close()

In [ ]:
rfiles = []
for tx in txs:
    f = '/home/lindb/teakettle/postmoran/by_tx/%s/%s_combine_R_vector_full_geno.R' % (tx,tx)
    rfiles.append(f)

shfiles = []
for f in rfiles:
    tx = op.basename(op.dirname(f))
    text ='''#!/bin/bash
#$ -N comb%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (tx,op.dirname(f),f)
    DIR = '/home/lindb/teakettle/postmoran/by_tx/shfiles/%s' % tx
    if not op.exists(DIR):
        mkdir(DIR)
    filE = op.join(DIR,'%s.sh' % tx)
    shfiles.append(filE)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
for s in shfiles:
    cd(op.dirname(s))
    !qsub $s

In [ ]:
# scp these files to computer
files = [f for f in fs('/home/lindb/teakettle/postmoran/by_tx/final/') if 'autogdgen' in f]
[files.append(f) for f in fs('/home/lindb/teakettle/postmoran/by_tx/final/') if 'bingdgen' in f]
for tx in txs:
    f = '/home/lindb/teakettle/postmoran/by_tx/%s/%s_full_geno_null.RDS' % (tx,tx)
    assert op.exists(f)
    files.append(f)
files

In [ ]:
### Put into R
```R
source('~/imports.R')
library(PopGenReport)
library(adegenet)
cd('~/Downloads/by_tx_full_geno')
getrds()
txs <- c('bc','uc','un','us')

Lows <- list()
His  <- list()
for (tx in txs){
    null <- get(sprintf('%s_full_geno_null',tx))
    h <- c()
    l <- c()
    for (i in 1:len(null)){
        h <- c(h,null[[i]][1])
        l <- c(l,null[[i]][2])
    }
    Lows <- append(Lows,list(l))
    His  <- append(His,list(h))
    names(Lows)[len(Lows)] <- tx
    names(His)[len(His)]   <- tx
}
```
### then use code below to plot figs

In [ ]:
# print out text to paste into R
for tx in txs:
    text = '''
tx = '%s'
his  <- His[[sprintf('%%s',tx)]]
his[which(is.na(his))] <- 0
lows <- Lows[[sprintf('%%s',tx)]]
lows[which(is.na(lows))] <- 0
sauto <- get(sprintf('autogdgen%%s',tx))
sauto[which(is.na(sauto$r)),'r'] <- 0
plot(sauto$bin,sauto$r,
    type="l",
    xlab="Distance (meters)",
    ylab="r",
    ylim=c(-0.13,0.13),
    xaxt="n",
    main=sprintf('%%s',tx))
xtick <- seq(0,max(sauto$bin),100)
axis(side=1,at=xtick)
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col='grey75',border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)''' % tx
    print text